In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from PIL import Image

import torch as nn
import cv2

---

## Dataset

In [21]:
string = "children_s games, adults playing (e.g., hopscotch, 4-square, dodgeball, playground apparatus, t-ball"

first_br = string.find(r"(")
if (first_br > 0):
    second_br = string.find(r")")
    if second_br > 0:
        string = string[:first_br] + string[second_br+1:]
    else:
        string = string[:first_br]

[sub.strip() for sub in string.split(',')]

['children_s games', 'adults playing']

In [ ]:
from torch.utils.data import Dataset


class BodyKeypointsDataset(Dataset):
    def __init__(self, csv_path, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if nn.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = cv2.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks], dtype=float).reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample